In [47]:
import eia
import pandas as pd
import requests
import json

import urllib3
from urllib.request import urlopen

### Data Retrieval through API connector to EIA

In [48]:
eia_key = "340d4d6e61d32840dd8d66e2185a675c"

In [49]:
def eia_search(query):
    api_key = eia_key
    url = 'http://api.eia.gov/series/?api_key=' + api_key
    series_id = query.replace(' ', '%20')
    final_url = url + "&series_id=" + series_id
    json_obj = urlopen(final_url)
    return json.load(json_obj)

In [50]:
eia_search('PET.RWTC.D')

{'request': {'command': 'series', 'series_id': 'PET.RWTC.D'},
 'series': [{'copyright': 'Thomson-Reuters',
   'data': [['20180521', 72.26],
    ['20180518', 71.23],
    ['20180517', 71.47],
    ['20180516', 71.43],
    ['20180515', 71.34],
    ['20180514', 71.01],
    ['20180511', 70.69],
    ['20180510', 71.36],
    ['20180509', 71.16],
    ['20180508', 68.83],
    ['20180507', 70.74],
    ['20180504', 69.71],
    ['20180503', 68.45],
    ['20180502', 67.91],
    ['20180501', 67.28],
    ['20180430', 68.56],
    ['20180427', 68.11],
    ['20180426', 68.18],
    ['20180425', 68],
    ['20180424', 67.66],
    ['20180423', 67.61],
    ['20180420', 68.26],
    ['20180419', 68.3],
    ['20180418', 68.44],
    ['20180417', 66.5],
    ['20180416', 66.23],
    ['20180413', 67.35],
    ['20180412', 67.07],
    ['20180411', 66.81],
    ['20180410', 65.48],
    ['20180409', 63.4],
    ['20180406', 62.03],
    ['20180405', 63.53],
    ['20180404', 63.35],
    ['20180403', 63.41],
    ['20180402',

In [51]:
data = eia_search('PET.RWTC.D')

### Data Transformation from list to data frame

In [52]:
data_list = data['series'][0]['data']

In [53]:
df = pd.DataFrame(data_list)

### Renaming Columns to Date and Oil Price

In [54]:
df.columns = ['date', 'oil_price']

### Setting date-time structure to Date variable

In [32]:
df['date'] = pd.to_datetime(df['date'])

### Importing Keras-Tensorflow Libraries

In [55]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

### Importing Train-Test Libraries

In [61]:
import math
import numpy
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### Normalizing Time-series Data

In [57]:
scaler = MinMaxScaler(feature_range=(0,1))

In [58]:
dataset = scaler.fit_transform(df)

### Split into Train and Test datasets

In [60]:
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

5717 2451


### Creating Time-lagged (period = 1) column

In [62]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [64]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [65]:
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

### Building Model

In [85]:
model = Sequential()

model.add(LSTM(
    input_shape=(None, 1),
    units=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    units=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')

print ('compilation time:'), time.time() - start

compilation time:


(None, 0.027415037155151367)

### Training Model

In [87]:
model.fit(
    trainX,
    trainY,
    batch_size=512,
    epochs=100,
    validation_split=0.05,
    verbose=2)

/Users/Weiyang/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 5429 samples, validate on 286 samples
Epoch 1/100
 - 3s - loss: 0.3698 - val_loss: 0.0278
Epoch 2/100
 - 0s - loss: 0.1910 - val_loss: 6.8446e-04
Epoch 3/100
 - 0s - loss: 0.0661 - val_loss: 0.0132
Epoch 4/100
 - 0s - loss: 0.0183 - val_loss: 0.0355
Epoch 5/100
 - 0s - loss: 0.0122 - val_loss: 0.0357
Epoch 6/100
 - 0s - loss: 0.0115 - val_loss: 0.0300
Epoch 7/100
 - 0s - loss: 0.0099 - val_loss: 0.0243
Epoch 8/100
 - 0s - loss: 0.0081 - val_loss: 0.0204
Epoch 9/100
 - 0s - loss: 0.0065 - val_loss: 0.0089
Epoch 10/100
 - 0s - loss: 0.0055 - val_loss: 0.0047
Epoch 11/100
 - 0s - loss: 0.0048 - val_loss: 0.0043
Epoch 12/100
 - 0s - loss: 0.0042 - val_loss: 0.0025
Epoch 13/100
 - 0s - loss: 0.0043 - val_loss: 0.0027
Epoch 14/100
 - 0s - loss: 0.0038 - val_loss: 0.0013
Epoch 15/100
 - 0s - loss: 0.0039 - val_loss: 0.0021
Epoch 16/100
 - 0s - loss: 0.0038 - val_loss: 7.2749e-04
Epoch 17/100
 - 0s - loss: 0.0037 - val_loss: 0.0012
Epoch 18/100
 - 0s - loss: 0.0037 - val_loss: 6.5852e

### Testing Model

In [88]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [89]:
print(trainPredict)

[[ 1.00393939]
 [ 1.00392962]
 [ 1.00392652]
 ..., 
 [ 0.29758152]
 [ 0.29757866]
 [ 0.29757577]]


### Inverting predictions back to original form

In [90]:
trainPredict = scaler.inverse_transform(trainPredict)

ValueError: non-broadcastable output operand with shape (5715,1) doesn't match the broadcast shape (5715,2)

In [83]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])


testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

ValueError: non-broadcastable output operand with shape (5715,1) doesn't match the broadcast shape (5715,2)

In [72]:
predictions = lstm.predict_sequences_multiple(model, testX, 3, 3)
lstm.plot_results_multiple(predictions, testY, 3)

NameError: name 'lstm' is not defined